# Hugging Face demo

In [1]:
import os
import time
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

from huggingface_hub import login
from huggingface_hub import HfApi
from transformers import pipeline
token = os.environ.get("HUGGINGFACE_TOKEN")
login(token=token)
#api = HfApi(token=token)

import torch
print(torch.mps.is_available())
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(device)

True
mps


## Import data

In [2]:
df = pd.read_csv("flu_vaccine_test.csv")
df_factual = pd.read_csv("flu_vaccine_test_factual.csv")
df_cf = pd.read_csv("flu_vaccine_test_counterfactual.csv")

## Process data

In [3]:
pairs = df_cf["Cause-effect pair"].unique()
n = 3
dfs = []
for pair in pairs:
    df_pair = df_cf[df_cf["Cause-effect pair"] == pair]
    
    cf_context = df_pair["Context"].to_numpy()
    cf_1_q = df_pair["Question (cause = True)"].to_numpy()
    cf_1_true = df_pair["True (cause = True)"].to_numpy()
    cf_0_q = df_pair["Question (cause = False)"].to_numpy()
    cf_0_true = df_pair["True (cause = False)"].to_numpy()
    
    cf_context = np.random.choice(cf_context, size = n)
    cf_1_q = np.random.choice(cf_1_q, size = n)
    cf_1_true = np.random.choice(cf_1_true, size = n)
    cf_0_q = np.random.choice(cf_0_q, size = n)
    cf_0_true = np.random.choice(cf_0_true, size = n)

    df_subset = pd.DataFrame({"Context": cf_context, 
                              "Question (cause = True)": cf_1_q, 
                              "True (cause = True)": cf_1_true, 
                              "Question (cause = False)": cf_0_q, 
                              "True (cause = False)": cf_0_true})
    df_subset["Cause-effect pair"] = pair
    dfs.append(df_subset)

df_subsets = pd.concat(dfs)
display(df_subsets)

,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False),Cause-effect pair
0,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",1,"Now, suppose that Trinidad did not get vaccina...",1,"('Trinidad', 'Harriet')"
1,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",1,"Now, suppose that Trinidad did not get vaccina...",1,"('Trinidad', 'Harriet')"
2,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",1,"Now, suppose that Trinidad did not get vaccina...",1,"('Trinidad', 'Harriet')"
0,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",0,"Now, suppose that Trinidad did not get vaccina...",0,"('Trinidad', 'Attie')"
1,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",1,"Now, suppose that Trinidad did not get vaccina...",0,"('Trinidad', 'Attie')"
2,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",0,"Now, suppose that Trinidad did not get vaccina...",0,"('Trinidad', 'Attie')"
0,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",1,"Now, suppose that Trinidad did not get vaccina...",1,"('Trinidad', 'Jazmin')"
1,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",0,"Now, suppose that Trinidad did not get vaccina...",0,"('Trinidad', 'Jazmin')"
2,A group of friends is considering whether or n...,"Now, suppose that Trinidad got vaccinated rega...",0,"Now, suppose that Trinidad did not get vaccina...",1,"('Trinidad', 'Jazmin')"
0,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')"


In [4]:
idx = [0,29995,14,21,154,107]
cf_pairs = df_cf["Cause-effect pair"].to_numpy()[idx]
cf_context = df_cf["Context"].to_numpy()[idx]
cf_1_q = df_cf["Question (cause = True)"].to_numpy()[idx]
cf_1_true = df_cf["True (cause = True)"].to_numpy()[idx]
cf_0_q = df_cf["Question (cause = False)"].to_numpy()[idx]
cf_0_true = df_cf["True (cause = False)"].to_numpy()[idx]

df_subset = pd.DataFrame({"Context": cf_context, 
                          "Question (cause = True)": cf_1_q, 
                          "True (cause = True)": cf_1_true, 
                          "Question (cause = False)": cf_0_q, 
                          "True (cause = False)": cf_0_true,
                          "Cause-effect pair": cf_pairs})

df_test = pd.concat([df_subset,df_subsets])
df_test = df_test.sort_values(by = 'Cause-effect pair').reset_index(drop = True)
df_test

,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False),Cause-effect pair
0,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')"
1,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')"
2,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')"
3,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')"
4,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')"
5,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')"
6,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')"
7,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')"
8,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')"
9,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')"


In [5]:
len(df_subsets)

18

In [6]:
f_context = df_factual["Context"].to_numpy()
f_q = df_factual["Question"].to_numpy()
f_true = df_factual["True"].to_numpy()

## Phi-4-mini-reasoning

In [7]:
# Use a pipeline as a high-level helper.
start = time.time()
pipe = pipeline("text-generation", model = "microsoft/Phi-4-mini-reasoning")
print("Model loading time:", round((time.time()-start)/60, 3))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0


Model loading time: 1.242


In [8]:
str1 = "A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated or if Amira gets vaccinated. Jayda will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season and Jazmin gets vaccinated. Arly will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season or if Jayda gets vaccinated. Harriet will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season or if Jazmin gets vaccinated or if Arly gets vaccinated.   During the previous flu season, Trinidad had stomach pain for 1 days, a low-grade fever for 3 days, a high fever for 6 days, bronchitis for 7 days, a cough for 4 days, and a runny nose for 7 days. Cheryle had a cough for 1 days, a high fever for 5 days, a runny nose for 6 days, bronchitis for 5 days, a low-grade fever for 3 days, and stomach pain for 3 days. Tosha had bronchitis for 3 days, a runny nose for 6 days, a low-grade fever for 8 days, a cough for 5 days, a high fever for 6 days, and stomach pain for 5 days. Attie had a runny nose for 2 days, a cough for 3 days, bronchitis for 3 days, a high fever for 3 days, stomach pain for 9 days, and a low-grade fever for 7 days. Danette had bronchitis for 9 days, stomach pain for 4 days, a high fever for 8 days, a runny nose for 5 days, a low-grade fever for 6 days, and a cough for 3 days. Amira had a runny nose for 2 days, a cough for 4 days, bronchitis for 7 days, stomach pain for 8 days, a low-grade fever for 8 days, and a high fever for 8 days. Jazmin had bronchitis for 3 days, a cough for 1 days, a high fever for 6 days, a low-grade fever for 7 days, stomach pain for 1 days, and a runny nose for 8 days. Jayda had bronchitis for 6 days, a runny nose for 3 days, a cough for 9 days, a low-grade fever for 1 days, a high fever for 6 days, and stomach pain for 4 days. Arly had bronchitis for 7 days, a high fever for 7 days, stomach pain for 6 days, a cough for 1 days, a low-grade fever for 3 days, and a runny nose for 9 days. Harriet had a runny nose for 3 days, a high fever for 4 days, bronchitis for 2 days, a low-grade fever for 4 days, stomach pain for 2 days, and a cough for 4 days.  Now, suppose that Attie got vaccinated regardless of all other circumstances. With this new assumption, did Harriet get vaccinated? Begin your response with Yes or No and be as concise as possible."

In [9]:
str2 = "A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated or if Amira gets vaccinated. Jayda will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season and Jazmin gets vaccinated. Arly will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season or if Jayda gets vaccinated. Harriet will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season or if Jazmin gets vaccinated or if Arly gets vaccinated.   During the previous flu season, Trinidad had stomach pain for 1 days, a low-grade fever for 3 days, a high fever for 6 days, bronchitis for 7 days, a cough for 4 days, and a runny nose for 7 days. Cheryle had a cough for 1 days, a high fever for 5 days, a runny nose for 6 days, bronchitis for 5 days, a low-grade fever for 3 days, and stomach pain for 3 days. Tosha had bronchitis for 3 days, a runny nose for 6 days, a low-grade fever for 8 days, a cough for 5 days, a high fever for 6 days, and stomach pain for 5 days. Attie had a runny nose for 2 days, a cough for 3 days, bronchitis for 3 days, a high fever for 3 days, stomach pain for 9 days, and a low-grade fever for 7 days. Danette had bronchitis for 9 days, stomach pain for 4 days, a high fever for 8 days, a runny nose for 5 days, a low-grade fever for 6 days, and a cough for 3 days. Amira had a runny nose for 2 days, a cough for 4 days, bronchitis for 7 days, stomach pain for 8 days, a low-grade fever for 8 days, and a high fever for 8 days. Jazmin had bronchitis for 3 days, a cough for 1 days, a high fever for 6 days, a low-grade fever for 7 days, stomach pain for 1 days, and a runny nose for 8 days. Jayda had bronchitis for 6 days, a runny nose for 3 days, a cough for 9 days, a low-grade fever for 1 days, a high fever for 6 days, and stomach pain for 4 days. Arly had bronchitis for 7 days, a high fever for 7 days, stomach pain for 6 days, a cough for 1 days, a low-grade fever for 3 days, and a runny nose for 9 days. Harriet had a runny nose for 3 days, a high fever for 4 days, bronchitis for 2 days, a low-grade fever for 4 days, stomach pain for 2 days, and a cough for 4 days. Now, suppose that Attie did not get vaccinated regardless of all other circumstances. With this new assumption, did Harriet get vaccinated? Begin your response with Yes or No and be as concise as possible."

In [10]:

start = time.time()
messages = [
    {"role": "user", "content": str1},
    {"role": "user", "content": str2},
]
response = pipe(messages)
print(response)
print("Inference time:", round((time.time()-start)/60, 3))

[{'generated_text': [{'role': 'user', 'content': 'A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin wi

In [11]:
response[0].get("generated_text")[1].get("content")

'A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin will get vaccinated if she had a high fever or bron

## DeepSeek R1

In [12]:
'''
start = time.time()
pipe_r1 = pipeline("text-generation", model = "deepseek-ai/DeepSeek-R1", trust_remote_code = True)
print("Model loading time:", round((time.time()-start)/60, 3))
'''

'\nstart = time.time()\npipe_r1 = pipeline("text-generation", model = "deepseek-ai/DeepSeek-R1", trust_remote_code = True)\nprint("Model loading time:", round((time.time()-start)/60, 3))\n'

In [ ]:
start = time.time()
messages = [
    {"role": "user", "content": f_context[0] + f_q[0]},
]
response = pipe_r1(messages)
print(response)
print("Inference time:", round((time.time()-start)/60, 3))

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct")
pipe(messages)

## o4-mini

In [46]:
for i,row in df_test.iterrows():
    print("\n----- {} -----".format(i))
    print("\n", row["Context"])
    print("\n", row["Question (cause = True)"])
    print("\n", row["Question (cause = False)"])


----- 0 -----

 A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin will get vaccinated if she had a hi

### Results

In [74]:
o4_mini_cf_1 = ["Yes—Attie’s vaccination makes Jazmin (and Arly) vaccinated, so Harriet meets her rule.",
                "Yes—Attie’s vaccination triggers Jazmin and Arly to vaccinate, satisfying Harriet’s OR rule.",
                "Yes—Attie’s vaccination triggers Jazmin to vaccinate, satisfying Harriet’s OR condition.",
                "Yes.",
                "Yes—Attie’s vaccination satisfies Jazmin’s OR condition, so she gets vaccinated.",
                "Yes—Jazmin had 9 days of bronchitis (≥7 days), so she gets vaccinated.",
                "Yes—Jazmin had 7 days of high fever (and 7 days of bronchitis), satisfying her ≥7‑day rule (and Attie is vaccinated).",
                "Yes—Attie’s vaccination satisfies Jazmin’s OR condition, so she gets vaccinated.",
                "Yes—Jazmin’s vaccination alone satisfies Harriet’s OR rule, so Harriet gets vaccinated.",
                "Yes—Jazmin’s vaccination alone satisfies Harriet’s OR condition, so she gets vaccinated.",
                "Yes—Jazmin’s vaccination alone satisfies Harriet’s OR rule, so she gets vaccinated.",
                "Yes—Jazmin’s vaccination alone satisfies Harriet’s OR condition, so she gets vaccinated.",
                "No—Tosha isn’t vaccinated (Cheryle isn’t), so Attie’s AND condition fails.",
                "No—Attie’s high fever (2 days) and bronchitis (3 days) are below the ≥5‑day threshold, and Tosha isn’t vaccinated.",
                "No—Attie had only 3 days of high fever and 2 days of bronchitis (<5 days), so she doesn’t get vaccinated.",
                "Yes—Attie had ≥5 days of illness and both Trinidad and Tosha vaccinate, so Attie does too.",
                "Yes—Jazmin vaccinates, satisfying Harriet’s OR condition.",
                "Yes—Arly vaccinates, satisfying Harriet’s OR condition.",
                "Yes—Harriet’s 5 days of high fever (and 6 days of bronchitis) meets her ≥5‑day rule, so she gets vaccinated.",
                "Yes—Harriet’s own high fever (6 days) and bronchitis (7 days) exceed the ≥5‑day threshold, so she gets vaccinated.",
                "Yes—her 8 days of bronchitis (and 7 days of high fever) meet the ≥7‑day rule, so she gets vaccinated.",
                "Yes—she had 8 days of high fever (and bronchitis), exceeding the ≥7‑day threshold.",
                "Yes—Jazmin’s 9 days of bronchitis exceed the ≥7‑day threshold, so she gets vaccinated.",
                "Yes—Jazmin had 9 days of high fever (and 7 days of bronchitis), meeting her ≥7‑day rule, so she gets vaccinated."]

print(len(o4_mini_cf_1))

24


In [75]:
o4_mini_cf_0 = ["Yes—Jazmin gets vaccinated, which activates Harriet’s rule.", 
                "No—Harriet’s own counts are below 5 days and neither Jazmin nor Arly vaccinate.",
                "Yes—Jazmin gets vaccinated, so Harriet does as well.",
                "Yes—Harriet’s high fever lasted 6 days (and bronchitis 5 days), exceeding her ≥5 day threshold.",
                "No—Jazmin’s own fever/bronchitis days are below 7, Attie isn’t vaccinated, and Amira also fails, so she doesn’t.",
                "Yes—Jazmin’s 9 days of bronchitis exceeds the ≥7 day threshold, so she gets vaccinated.",
                "Yes—her 7 days of high fever (and bronchitis) meet the ≥7‑day threshold, so she gets vaccinated.",
                "Yes—Jazmin’s 7 days of high fever (and bronchitis) meets her ≥7‑day rule, so she gets vaccinated.",
                "Yes—Harriet’s 7 days of high fever (and 6 days of bronchitis) exceed the ≥5‑day threshold, so she gets vaccinated.",
                "Yes—Harriet’s 5 days of high fever (and 6 days of bronchitis) meets her ≥5‑day criterion, so she gets vaccinated.",
                "No—Harriet’s own fever/bronchitis days (3 days each) are below 5 days, and with Jazmin and Arly unvaccinated, she doesn’t.",
                "Yes—Arly meets her ≥6 day threshold (7 days of high fever) and vaccinates, satisfying Harriet’s OR condition.",
                "No—Attie’s rule requires Trinidad to be vaccinated, which she isn’t, so Attie does not.",
                "No—Attie’s rule requires Trinidad to be vaccinated, which she isn’t, so Attie does not.",
                "No—Attie’s rule requires Trinidad to be vaccinated, which she isn’t, so Attie does not.",
                "No—Attie doesn’t get vaccinated, since Trinidad isn’t vaccinated.",
                "No—Attie’s rule requires Trinidad (and Tosha) to vaccinate, and Trinidad doesn’t, so Attie does not.",
                "Yes—Arly vaccinates (high fever 8 days), satisfying Harriet’s OR condition.",
                "Yes.",
                "Yes—Harriet’s 6 days of high fever (and 7 days of bronchitis) exceed the 5‑day threshold, so she gets vaccinated.",
                "Yes—her 8 days of bronchitis (and 7 days of high fever) meet the ≥7‑day rule, so she gets vaccinated.",
                "Yes—Jazmin’s 8 days of high fever (and bronchitis) exceed the ≥7‑day rule, so she gets vaccinated.",
                "Yes—her 9 days of bronchitis exceed the ≥7‑day threshold, so she gets vaccinated.",
                "Yes—Jazmin had 9 days of high fever (and 7 days of bronchitis), exceeding her ≥7‑day threshold, so she gets vaccinated."]

print(len(o4_mini_cf_0))

24


In [77]:
df_test["o4-mini (cause = True)"] = o4_mini_cf_1
df_test["o4-mini (cause = False)"] = o4_mini_cf_0
df_test

,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False),Cause-effect pair,o4-mini (cause = True),o4-mini (cause = False)
0,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination makes Jazmin (and Arly...,"Yes—Jazmin gets vaccinated, which activates Ha..."
1,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin and Ar...,No—Harriet’s own counts are below 5 days and n...
2,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin to vac...,"Yes—Jazmin gets vaccinated, so Harriet does as..."
3,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes.,Yes—Harriet’s high fever lasted 6 days (and br...
4,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,No—Jazmin’s own fever/bronchitis days are belo...
5,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')","Yes—Jazmin had 9 days of bronchitis (≥7 days),...",Yes—Jazmin’s 9 days of bronchitis exceeds the ...
6,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Jazmin had 7 days of high fever (and 7 day...,Yes—her 7 days of high fever (and bronchitis) ...
7,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,Yes—Jazmin’s 7 days of high fever (and bronchi...
8,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 7 days of high fever (and 6 days...
9,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 5 days of high fever (and 6 days...


In [78]:
o4_mini_cf_1_bool = []
for r in o4_mini_cf_1:
    if r[:3] == "Yes":
        o4_mini_cf_1_bool.append(1)
    elif r[:2] == "No":
        o4_mini_cf_1_bool.append(0)

o4_mini_cf_0_bool = []
for r in o4_mini_cf_0:
    if r[:3] == "Yes":
        o4_mini_cf_0_bool.append(1)
    elif r[:2] == "No":
        o4_mini_cf_0_bool.append(0)

print(o4_mini_cf_1_bool)
print(o4_mini_cf_0_bool)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [80]:
df_test["o4-mini bool (cause = True)"] = o4_mini_cf_1_bool
df_test["o4-mini bool (cause = False)"] = o4_mini_cf_0_bool
df_test["Concordant (cause = True)"] = np.where(df_test["True (cause = True)"] == df_test["o4-mini bool (cause = True)"], 1, 0)
df_test["Concordant (cause = False)"] = np.where(df_test["True (cause = False)"] == df_test["o4-mini bool (cause = False)"], 1, 0)
df_test

,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False),Cause-effect pair,o4-mini (cause = True),o4-mini (cause = False),o4-mini bool (cause = True),o4-mini bool (cause = False),Concordant (cause = True),Concordant (cause = False)
0,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination makes Jazmin (and Arly...,"Yes—Jazmin gets vaccinated, which activates Ha...",1,1,1,1
1,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin and Ar...,No—Harriet’s own counts are below 5 days and n...,1,0,1,1
2,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin to vac...,"Yes—Jazmin gets vaccinated, so Harriet does as...",1,1,1,1
3,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes.,Yes—Harriet’s high fever lasted 6 days (and br...,1,1,1,0
4,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,No—Jazmin’s own fever/bronchitis days are belo...,1,0,1,1
5,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')","Yes—Jazmin had 9 days of bronchitis (≥7 days),...",Yes—Jazmin’s 9 days of bronchitis exceeds the ...,1,1,1,0
6,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Jazmin had 7 days of high fever (and 7 day...,Yes—her 7 days of high fever (and bronchitis) ...,1,1,1,1
7,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,Yes—Jazmin’s 7 days of high fever (and bronchi...,1,1,1,1
8,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 7 days of high fever (and 6 days...,1,1,1,1
9,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 5 days of high fever (and 6 days...,1,1,1,1


In [7]:
df_test = pd.read_csv("o4_mini_results.csv")
df_test.columns = ["Index", "Context", "Question (cause = True)", "True (cause = True)",
                   "Question (cause = False)", "True (cause = False)", "Cause-effect pair",
                   "o4-mini (cause = True)", "o4-mini (cause = False)",	"o4-mini bool (cause = True)",
                   "o4-mini bool (cause = False)", "Concordant (cause = True)", "Concordant (cause = False)"]
df_test

,Index,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False),Cause-effect pair,o4-mini (cause = True),o4-mini (cause = False),o4-mini bool (cause = True),o4-mini bool (cause = False),Concordant (cause = True),Concordant (cause = False)
0,0,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination makes Jazmin (and Arly...,"Yes—Jazmin gets vaccinated, which activates Ha...",1,1,1,1
1,1,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin and Ar...,No—Harriet’s own counts are below 5 days and n...,1,0,1,1
2,2,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Harriet')",Yes—Attie’s vaccination triggers Jazmin to vac...,"Yes—Jazmin gets vaccinated, so Harriet does as...",1,1,1,1
3,3,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Harriet')",Yes.,Yes—Harriet’s high fever lasted 6 days (and br...,1,1,1,0
4,4,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,No—Jazmin’s own fever/bronchitis days are belo...,1,0,1,1
5,5,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",0,"('Attie', 'Jazmin')","Yes—Jazmin had 9 days of bronchitis (≥7 days),...",Yes—Jazmin’s 9 days of bronchitis exceeds the ...,1,1,1,0
6,6,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Jazmin had 7 days of high fever (and 7 day...,Yes—her 7 days of high fever (and bronchitis) ...,1,1,1,1
7,7,A group of friends is considering whether or n...,"Now, suppose that Attie got vaccinated regardl...",1,"Now, suppose that Attie did not get vaccinated...",1,"('Attie', 'Jazmin')",Yes—Attie’s vaccination satisfies Jazmin’s OR ...,Yes—Jazmin’s 7 days of high fever (and bronchi...,1,1,1,1
8,8,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 7 days of high fever (and 6 days...,1,1,1,1
9,9,A group of friends is considering whether or n...,"Now, suppose that Jazmin got vaccinated regard...",1,"Now, suppose that Jazmin did not get vaccinate...",1,"('Jazmin', 'Harriet')",Yes—Jazmin’s vaccination alone satisfies Harri...,Yes—Harriet’s 5 days of high fever (and 6 days...,1,1,1,1


In [8]:
print("F1        =", f1_score(df_test["True (cause = True)"], df_test["o4-mini bool (cause = True)"]))
print("Precision =", precision_score(df_test["True (cause = True)"], df_test["o4-mini bool (cause = True)"]))
print("Recall    =", recall_score(df_test["True (cause = True)"], df_test["o4-mini bool (cause = True)"]))
print("Accuracy  =", accuracy_score(df_test["True (cause = True)"], df_test["o4-mini bool (cause = True)"]))

F1        = 1.0
Precision = 1.0
Recall    = 1.0
Accuracy  = 1.0


In [10]:
print("F1        =", f1_score(df_test["True (cause = False)"], df_test["o4-mini bool (cause = False)"]))
print("Precision =", precision_score(df_test["True (cause = False)"], df_test["o4-mini bool (cause = False)"]))
print("Recall    =", recall_score(df_test["True (cause = False)"], df_test["o4-mini bool (cause = False)"]))
print("Accuracy  =", accuracy_score(df_test["True (cause = False)"], df_test["o4-mini bool (cause = False)"]))

F1        = 0.8666666666666667
Precision = 0.8125
Recall    = 0.9285714285714286
Accuracy  = 0.8333333333333334


In [16]:
string = "A group of friends is considering whether or not to get vaccinated against the flu this year. Trinidad will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season. Cheryle will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Trinidad gets vaccinated. Tosha will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Cheryle gets vaccinated. Attie will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season and Trinidad gets vaccinated and Tosha gets vaccinated. Danette will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated. Amira will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season and Danette gets vaccinated. Jazmin will get vaccinated if she had a high fever or bronchitis for at least 7 days in the previous flu season or if Attie gets vaccinated or if Amira gets vaccinated. Jayda will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season and Jazmin gets vaccinated. Arly will get vaccinated if she had a high fever or bronchitis for at least 6 days in the previous flu season or if Jayda gets vaccinated. Harriet will get vaccinated if she had a high fever or bronchitis for at least 5 days in the previous flu season or if Jazmin gets vaccinated or if Arly gets vaccinated. During the previous flu season, Trinidad had a runny nose for 9 days, a high fever for 7 days, stomach pain for 2 days, a cough for 8 days, bronchitis for 7 days, and a low-grade fever for 2 days. Cheryle had stomach pain for 6 days, a low-grade fever for 4 days, a high fever for 9 days, a cough for 9 days, bronchitis for 7 days, and a runny nose for 6 days. Tosha had bronchitis for 5 days, a low-grade fever for 8 days, stomach pain for 1 days, a runny nose for 4 days, a cough for 3 days, and a high fever for 3 days. Attie had a low-grade fever for 5 days, a runny nose for 1 days, bronchitis for 3 days, stomach pain for 8 days, a high fever for 2 days, and a cough for 8 days. Danette had stomach pain for 3 days, a cough for 9 days, a high fever for 8 days, a low-grade fever for 2 days, a runny nose for 4 days, and bronchitis for 8 days. Amira had a low-grade fever for 5 days, stomach pain for 4 days, bronchitis for 6 days, a cough for 7 days, a high fever for 3 days, and a runny nose for 6 days. Jazmin had bronchitis for 8 days, stomach pain for 5 days, a runny nose for 1 days, a cough for 9 days, a high fever for 7 days, and a low-grade fever for 9 days. Jayda had a cough for 6 days, a runny nose for 1 days, bronchitis for 4 days, a low-grade fever for 2 days, a high fever for 5 days, and stomach pain for 2 days. Arly had stomach pain for 7 days, bronchitis for 6 days, a high fever for 6 days, a runny nose for 2 days, a low-grade fever for 3 days, and a cough for 5 days. Harriet had stomach pain for 7 days, a high fever for 6 days, a cough for 5 days, bronchitis for 7 days, a low-grade fever for 3 days, and a runny nose for 2 days. "

In [17]:
df_cf_c = df_cf[df_cf["Context"] == string]
df_cf_c

,Task ID,Context ID,Sample ID,Nodes per BCC,Cause-effect pair,Cause,Effect,Context,Question (cause = True),True (cause = True),Question (cause = False),True (cause = False)


In [19]:
idx = [0,29995,14,21,154,107]

In [28]:
from utils import Utils

ModuleNotFoundError: No module named 'matplotlib'

In [27]:
DAG adjacency matrix

,Task ID,Context ID,Sample ID,Nodes per BCC,DAG adjacency matrix,DAG nodes,CCT adjacency matrix,CCT nodes,Exogenous variables,Bernoulli parameters,Global quantity,Local quantities,Compositions,Causal context,Sample context,Factual queries,Interventional queries (cause = True),Interventional queries (cause = False)
0,0.0000,0,0,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 0 0 0 0 0 0 0]...,"['Trinidad', 'Cheryle', 'Tosha', 'Attie', 'Dan...",[[0 1 1 1]\n [0 0 1 1]\n [0 0 0 1]\n [0 0 0 0]],"['Trinidad', 'Attie', 'Jazmin', 'Harriet']","['OWE6', 'JKOY', 'KFRA', 'JJ13', 'EXOD', '6PH9...","[0.5, 0.7, 0.7, 0.5, 0.7, 0.7, 0.7, 0.6, 0.6, ...","('Trinidad', 'Harriet')","[('Trinidad', 'Attie'), ('Trinidad', 'Jazmin')...","[[('Trinidad', 'Attie'), ('Attie', 'Jazmin'), ...",A group of friends is considering whether or n...,"During the previous flu season, Trinidad had ...",{'Harriet': {'Prompt': 'Did Harriet get vaccin...,"{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup...","{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup..."
1,0.1000,0,1,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 0 0 0 0 0 0 0]...,"['Trinidad', 'Cheryle', 'Tosha', 'Attie', 'Dan...",[[0 1 1 1]\n [0 0 1 1]\n [0 0 0 1]\n [0 0 0 0]],"['Trinidad', 'Attie', 'Jazmin', 'Harriet']","['OWE6', 'JKOY', 'KFRA', 'JJ13', 'EXOD', '6PH9...","[0.5, 0.7, 0.7, 0.5, 0.7, 0.7, 0.7, 0.6, 0.6, ...","('Trinidad', 'Harriet')","[('Trinidad', 'Attie'), ('Trinidad', 'Jazmin')...","[[('Trinidad', 'Attie'), ('Attie', 'Jazmin'), ...",A group of friends is considering whether or n...,"During the previous flu season, Trinidad had ...",{'Harriet': {'Prompt': 'Did Harriet get vaccin...,"{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup...","{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup..."
2,0.2000,0,2,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 0 0 0 0 0 0 0]...,"['Trinidad', 'Cheryle', 'Tosha', 'Attie', 'Dan...",[[0 1 1 1]\n [0 0 1 1]\n [0 0 0 1]\n [0 0 0 0]],"['Trinidad', 'Attie', 'Jazmin', 'Harriet']","['OWE6', 'JKOY', 'KFRA', 'JJ13', 'EXOD', '6PH9...","[0.5, 0.7, 0.7, 0.5, 0.7, 0.7, 0.7, 0.6, 0.6, ...","('Trinidad', 'Harriet')","[('Trinidad', 'Attie'), ('Trinidad', 'Jazmin')...","[[('Trinidad', 'Attie'), ('Attie', 'Jazmin'), ...",A group of friends is considering whether or n...,"During the previous flu season, Trinidad had ...",{'Harriet': {'Prompt': 'Did Harriet get vaccin...,"{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup...","{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup..."
3,0.3000,0,3,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 0 0 0 0 0 0 0]...,"['Trinidad', 'Cheryle', 'Tosha', 'Attie', 'Dan...",[[0 1 1 1]\n [0 0 1 1]\n [0 0 0 1]\n [0 0 0 0]],"['Trinidad', 'Attie', 'Jazmin', 'Harriet']","['OWE6', 'JKOY', 'KFRA', 'JJ13', 'EXOD', '6PH9...","[0.5, 0.7, 0.7, 0.5, 0.7, 0.7, 0.7, 0.6, 0.6, ...","('Trinidad', 'Harriet')","[('Trinidad', 'Attie'), ('Trinidad', 'Jazmin')...","[[('Trinidad', 'Attie'), ('Attie', 'Jazmin'), ...",A group of friends is considering whether or n...,"During the previous flu season, Trinidad had ...",{'Harriet': {'Prompt': 'Did Harriet get vaccin...,"{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup...","{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup..."
4,0.4000,0,4,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 0 0 0 0 0 0 0]...,"['Trinidad', 'Cheryle', 'Tosha', 'Attie', 'Dan...",[[0 1 1 1]\n [0 0 1 1]\n [0 0 0 1]\n [0 0 0 0]],"['Trinidad', 'Attie', 'Jazmin', 'Harriet']","['OWE6', 'JKOY', 'KFRA', 'JJ13', 'EXOD', '6PH9...","[0.5, 0.7, 0.7, 0.5, 0.7, 0.7, 0.7, 0.6, 0.6, ...","('Trinidad', 'Harriet')","[('Trinidad', 'Attie'), ('Trinidad', 'Jazmin')...","[[('Trinidad', 'Attie'), ('Attie', 'Jazmin'), ...",A group of friends is considering whether or n...,"During the previous flu season, Trinidad had ...",{'Harriet': {'Prompt': 'Did Harriet get vaccin...,"{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup...","{('Trinidad', 'Harriet'): {'Prompt': 'Now, sup..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.4995,0,4995,"[4, 4, 4]",[[0 1 0 1 0 0 0 0 0 0]\n [0 0 1 

In [29]:
df_factual

,Task ID,Context ID,Sample ID,Nodes per BCC,Effect,Context,Question,True
0,0.0000,0,0,"[4, 4, 4]",Harriet,A group of friends is considering whether or n...,Did Harriet get vaccinated? Begin your respons...,1
1,0.0000,0,0,"[4, 4, 4]",Attie,A group of friends is considering whether or n...,Did Attie get vaccinated? Begin your response ...,0
2,0.0000,0,0,"[4, 4, 4]",Jazmin,A group of friends is considering whether or n...,Did Jazmin get vaccinated? Begin your response...,1
3,0.1000,0,1,"[4, 4, 4]",Harriet,A group of friends is considering whether or n...,Did Harriet get vaccinated? Begin your respons...,1
4,0.1000,0,1,"[4, 4, 4]",Attie,A group of friends is considering whether or n...,Did Attie get vaccinated? Begin your response ...,0
...,...,...,...,...,...,...,...,...
14995,0.4998,0,4998,"[4, 4, 4]",Attie,A group of friends is considering whether or n...,Did Attie get vaccinated? Begin your response ...,0
14996,0.4998,0,4998,"[4, 4, 4]",Jazmin,A group of friends is considering whether or n...,Did Jazmin get vaccinated? Begin your response...,1
14997,0.4999,0,4999,"[4, 4, 4]",Harriet,A group of friends is considering whether or n...,Did Harriet get vaccinated? Begin your respons...,1
14998,0.4999,0,4999,"[4, 4, 4]",Attie,A group of friends is considering whether or n...,Did Attie get vaccinated? Begin your response ...,0


## End of document